<img src="img/logo_demcompare.png" width="100" align="right">

# Demcompare: statistics tutorial

This notebook is an introduction to demcompare and its statistic step.

## Imports and external functions

In [ ]:
import pyproj # pyproj as first import is necessary

In [ ]:
from snippets.utils_notebook import *

In [ ]:
from IPython.display import HTML, display
import tabulate

## What is Demcompare ? 

* Demcompare is a python software that aims at comparing two DEMs together.
* It performs the coregistration based on the Nuth & Kääb universal coregistration method.
* Two steps are available in DEMcompare coregistration's step: reprojection and coregistration (not shown in this Notebook)
* It provides a wide variety of standard metrics which can be classified.

**If just the statistics step is present on the input configuration:**

Demcompare can compute a wide variety of statistics on either an input DEM, or the difference between two input DEMs. The statistics module can consider different number of inputs:

* If one single DEM is specified in the configuration; in this case the input or default metrics will directly be computed on the input DEM.
* If two DEMs are specified in the configuration; demcompare will do the reprojection for both DEMs to have the same resolution and size, and the difference between both reprojected DEMs will be considered to compute the input or default metrics.

## Glossary

**DEM (Digital Elevation Model)** : a 3D computer graphics representation of elevation data to represent terrain.

### 1. For one single DEM

We first present the statistical step for one DEM.

You must load the DEM with the `load_dem` function

In [ ]:
from demcompare.dem_tools import load_dem

The DEM is configurated using a dictionnary

In [ ]:
dem_dict = {
            "path" : "data/grenoble/Copernicus_DSM_10_N45_00_E005_00_DEM.tif",
            "zunit" : "m",
            "nodata" : -32768
    }

Load dem with the following function

In [ ]:
dem = load_dem(
    path=dem_dict["path"], zunit=dem_dict["zunit"]
)

Import StatsProcessing class

In [ ]:
from demcompare.stats_processing import StatsProcessing

You can show all available metrics that can be computed with demcompare

In [ ]:
print(StatsProcessing.show_all_available_metrics())

If you want the statistics step to be calculated, you must instantiate the dictionary in the configuration.

In [ ]:
cfg = {"statistics":{}} 

Create the stats_processing object including configuration and DEM.

In [ ]:
stats_processing = StatsProcessing(cfg['statistics'], dem)

All available classification layer that will be computed by demcompare with your configuration:

In [ ]:
print(stats_processing.show_available_classification_layers())

#### 1.1. Global scalar metrics

* The global classification is a type of segmentation classification which is always computed by default.
* This layer has a single class where all valid pixels are considered. 
* If no classification layers are specified in the input configuration, only the global classification will be considered.
* All metrics here are scalar

Calculate metrics requested in the configuration and store the result in a StatsDataset object

In [ ]:
stats_dataset = stats_processing.compute_stats()

Here, we present the way to get the results for computed metrics.  

Get the name of all calculated metrics with `get_classification_layer_metrics`. The type of classification layer must be mentioned as an argument.

In [ ]:
stats_metrics = stats_dataset.get_classification_layer_metrics(classification_layer="global")
print(stats_metrics)

The metrics are always computed on valid pixels. Valid pixels are those whose value is different than NaN and the nodata value

Apart from only considering the valid pixels, the user may also specify the `remove_outliers` option in the input configuration. This option will also filter all DEM pixels outside (mu + 3 sigma) and (mu - 3 sigma), being mu the mean and sigma the standard deviation of all valid pixels in the DEM.

To get the metric's value you can use the `get_classification_layer_metric`. The type of classification layer and the metric name must be mentioned as an argument.

In [ ]:
list_metrics = [["Metric's name", "Measured metrics"]]
for metric in stats_metrics: 
    value = stats_dataset.get_classification_layer_metric(classification_layer="global", metric=metric)
    list_metrics.append([metric, value[0]])
    
display(HTML(tabulate.tabulate(list_metrics, tablefmt='html')))

#### 1.2. Classification layers

Classification layers are a way to classify the DEM pixels in classes according to different criteria in order to compute specific statistics according to each class. We have seen the global classification layer but there are three others.

##### 1.2.1. Slope

This type of classification computes the slope of the input DEMs and classifies the pixels according to the range on which its slope falls.

Import compute_dem_slope function

In [ ]:
from demcompare.dem_tools import compute_dem_slope

Compute slope and add it as a classification_layer

In [ ]:
slope_dem = compute_dem_slope(dem)

To use the slope classification layer for statistics, the user must specify the slope configuration in the statistics configuration. 

Note that you can specify the metric you need for this classification layer. Min and max for example.

In [ ]:
cfg = {
    "statistics": {
        "classification_layers": {
             "Slope0": {
                  "type": "slope",
                  "ranges": [0, 25, 45, 65],
                  "metrics": ["min", "max"]
             },
        }
    }
}

The StatsProcessing object is instantiate and the slope classification named Slope0 is computed on the new dem.

In [ ]:
stats_processing_ = StatsProcessing(cfg['statistics'], slope_dem)

Calculate metrics requested in the configuration and store the result in a StatsDataset object

In [ ]:
stats_dataset = stats_processing_.compute_stats()

The Slope0 classification layer is now available.

In [ ]:
stats_dataset.get_classification_layer_names()

Get all available metrics for this classification layer. nbpts and percent_valid_points are always calculated for classification layers. But we can also see the min and max metrics.

In [ ]:
stats_metrics = stats_dataset.get_classification_layer_metrics(classification_layer = 'Slope0')
print(stats_metrics)

Get all available classes for this classification layer.

In [ ]:
ranges_slope = list(stats_processing_.classification_layers[0].classes.values())

Display all measured values for Slope0 according to class.

In [ ]:
list_metrics = [["Metric name", "Measure"]]
list_metrics.append(["ranges slope", ranges_slope])
for metric in stats_metrics: 
    measure = stats_dataset.get_classification_layer_metric(classification_layer="Slope0", metric=metric)
    list_metrics.append([metric, measure])
    
display(HTML(tabulate.tabulate(list_metrics, tablefmt='html')))

Getting a special measure for Slope (ex: minimum for ranges [0%:4%[)

In [ ]:
print(stats_dataset.get_classification_layer_metric(classification_layer="Slope0", metric="min", classif_class=0))

Plot classification layers masks according to the class

In [ ]:
plot_layers(stats_processing_, 'ref', dem, 'Slope masks on DEM', 'Slope0')

##### 1.2.2. Segmentation

This type of classification layer considers an input classification mask in order to classify the DEM pixels. 

The classification mask must be specified with its classes, and linked to one of the input DEMs defined in the input configuration as follows:

In [ ]:
dem_dict = {
            "path" : "data/grenoble/Copernicus_DSM_10_N45_00_E005_00_DEM.tif",
            "zunit" : "m",
            "georef": "WGS84",
            "classification_layers": {
                "Status": {"map_path": "data/grenoble/copernicus_status.tif"}
            }
        }

Here we need to reload the DEM with the classification layer.

In [ ]:
dem = load_dem(path=dem_dict["path"],                     
                     zunit=dem_dict["zunit"], 
                     classification_layers=dem_dict["classification_layers"])

To use the segmentation classification layer for statistics, the user must specify the segmentation configuration in the statistics configuration.

Note that you can specify the metric you need for this classification layer. Mean for example.

In [ ]:
cfg = {
    "statistics": {
        "classification_layers": {
             "Status": {
                  "type": "segmentation",
                  "classes": {"no data": [0],"fields": [1],"cities": [2], "waters": [3],"forests": [4]},
                 "metrics" : ["mean"]
             }
        }
    }
}

StatsPair.init will generate the segmentation classification layers on the new dem.

In [ ]:
stats_processing_ = StatsProcessing(cfg['statistics'], dem)

Calculate metrics requested in the configuration and store the result in a StatsDataset object

In [ ]:
stats_dataset = stats_processing_.compute_stats()

The Status classification layer is now available.

In [ ]:
stats_dataset.get_classification_layer_names()

Get all available metrics for this classification layer.

In [ ]:
stats_metrics = stats_dataset.get_classification_layer_metrics(classification_layer = 'Status')
print(stats_metrics)

Get all available classes for this classification layer.

In [ ]:
ranges_seg = list(stats_processing_.classification_layers[0].classes.keys())

Display all measured values for the Status according to the class.

In [ ]:
list_metrics = [["Metric name", "Measure"]]
list_metrics.append(["Classes segmentation", ranges_seg])
for metric in stats_metrics: 
    measure = stats_dataset.get_classification_layer_metric(classification_layer="Status", metric=metric)
    list_metrics.append([metric, measure])

display(HTML(tabulate.tabulate(list_metrics, tablefmt='html')))

Plot classification layers mask according to the class

In [ ]:
plot_layers(stats_processing_, 'ref', dem, 'Classification layers from segmentation on DEM', 'Status')

##### 1.2.3. Fusion

* This type of classification layer is created from two or more existing classification layers, as it is the result of fusing the classes of different classification layers.
* It is to be noticed that only classification layers belonging to the same support DEM can be fused.

Fusion schema : 
<img src="img/stats_fusion_schema.png" width="800">

Here, we initiate the fusion step through the Status and Slope0 steps.

In [ ]:
cfg = {
    "statistics": {
        "classification_layers": {
             "Status": {
                  "type": "segmentation",
                  "classes": {"no data": [0],"fields": [1],"cities": [2], "waters": [3],"forests": [4]},
             },
             "Slope0": {
                  "type": "slope",
                  "ranges": [0, 25, 45, 65]
             },
             "Fusion0": {
                 "type" : "fusion",
                  "ref": ["Slope0", "Status"]
             }
        },
    }
}

Compute slope and add it as a classification_layer

In [ ]:
slope_dem = compute_dem_slope(dem)

StatsPair.init will generate the segmentation classification layers on the new dem.

In [ ]:
stats_processing_= StatsProcessing(cfg['statistics'], slope_dem)

Calculate metrics requested in the configuration and store the result in a StatsDataset object

In [ ]:
stats_dataset = stats_processing_.compute_stats()

The Fusion0 classification layer is now available.

In [ ]:
stats_dataset.get_classification_layer_names()

Here we can see the different classes available on the fusion classification. They are indeed a fusion between Status and Slope0's classes.

In [ ]:
plot_layers(stats_processing_, 'ref', dem, 'Value by segmentation and slope fusion on DEM', 'Fusion0')

#### 1.3. More notions about getting metrics

We saw that the user can compute metrics thanks to the configuration dictionnary. But he can also use an API to compute the wanted metrics.  

In [ ]:
stats_dataset = stats_processing_.compute_stats(metrics = ["max"], classification_layer = ['Slope0'])

In [ ]:
max_by_class = stats_dataset.get_classification_layer_metric(
            classification_layer="Slope0", metric="max"
        )

for cpt, i in enumerate(ranges_slope):
    print(f"Maximum for classe {ranges_slope[cpt]} is equal to {max_by_class[cpt]}")

### 2. For two DEM

We can also compute statistics in outputs of coregistration step or directly between two DEMs.

Steps schema for two DEMs :
<img src="img/stats_input_two_dems.png" width="800">

We need to load two DEMs and we can add some classification layers.

In [ ]:
input_ref_config = {
            "path" : "data/grenoble/Copernicus_DSM_10_N45_00_E005_00_DEM.tif",
            "zunit" : "m",
            "georef": "WGS84",
            "classification_layers": {
                "Status": {"map_path": "data/grenoble/copernicus_status.tif"}
            }
        }
input_sec_config = {
                    "path" : "data/grenoble/Copernicus_blurred_and_shifted.tif",
                    "zunit" : "m",
                    "georef": "EPSG:32630",
                    "nodata" : -32768,
                }

Loading the DEMs

In [ ]:
input_ref = load_dem(
    path=input_ref_config["path"], 
    zunit=input_ref_config["zunit"],
    classification_layers=input_ref_config["classification_layers"]
)

input_sec = load_dem(
    path=input_sec_config["path"], 
    zunit=input_sec_config["zunit"], 
)

We can see that there are differences in terms of size and resolution. But there is also an offset between them.

In [ ]:
show(side_by_side_fig(input_ref, 
                 input_sec,
                "Reference DEM", 
                "Second DEM"))

##### 2.1. Prerequisites for statistics computation

DEMs must have same size and resolution so we need to reproject them. 

In [ ]:
from demcompare.dem_tools import reproject_dems
reproj_sec, reproj_ref, _ = reproject_dems(input_sec, input_ref, sampling_source = "ref")

Computing the slope for both DEMs with `compute_dem_slope`

In [ ]:
reproj_ref = compute_dem_slope(reproj_ref)
reproj_sec = compute_dem_slope(reproj_sec)

Statistics must be computed on the altitude difference's DEM, which is computed with the function `compute_alti_diff_for_stats`

In [ ]:
from demcompare.dem_tools import compute_alti_diff_for_stats

Computing `compute_alti_diff_for_stats`

In [ ]:
altitude_diff = compute_alti_diff_for_stats(reproj_ref, reproj_sec)

In [ ]:
show_dem(altitude_diff, 
         "Altitude difference on reprojected DEM")

##### 2.2. Computing the stats on altitude_diff

A configuration of statistical steps with segmentation, slope and fusion.

In [ ]:
cfg = {
    "statistics": {
        "classification_layers": {
             "Status": {
                  "type": "segmentation",
                  "classes": {"no data": [0],"fields": [1],"cities": [2], "waters": [3],"forests": [4]}
             },
             "Slope0": {
                  "type": "slope",
                  "ranges": [0, 25, 45, 65]
             },
             "Fusion0": {
                "type" : "fusion",
                  "ref": ["Slope0", "Status"]
             }
        },
    }
}

Create object from `StatsProcessing` with configuration and computed altitudes differences. If the input dem is an altitude difference the input_diff parameter is set to True. 

In [ ]:
stats_processing_ = StatsProcessing(cfg['statistics'], altitude_diff, input_diff=True)

Calculate metrics requested in the configuration and store the result in a StatsDataset object

In [ ]:
stats_dataset = stats_processing_.compute_stats()

Get all available classification layers

In [ ]:
stats_metrics = stats_dataset.get_classification_layer_names()
print(stats_metrics)

##### 2.3. Metrics in global classification for two DEM

You can see that more default metrics are computed when two DEM are used in inputs, as some metrics only make sense when computed on a difference or an error DEM

In [ ]:
stats_metrics = stats_dataset.get_classification_layer_metrics(classification_layer="global")
print(stats_metrics)

To get the metric's value use `get_classification_layer_metric`, the type of classification layer and the metric name must be mentioned as an argument.

If the input dem is an altitude difference the RMSE and the NMAD scalar metrics are measured.

In [ ]:
list_metrics = [["Metric's name", "Measured metrics"]]
for metric in stats_metrics: 
    value = stats_dataset.get_classification_layer_metric(classification_layer="global", metric=metric)
    list_metrics.append([metric, value[0]])
    
display(HTML(tabulate.tabulate(list_metrics, tablefmt='html')))

##### 2.3.1. Global vector metrics

Demcompare computes by default only scalar metrics for the global layer. But demcompare can also generate vector metrics such as cdf, pdf or ratio_above_threshold if specified in the configuration or the API

We can directly compute them by adding them to the compute_stats function.

In [ ]:
stats_dataset = stats_processing_.compute_stats(metrics = ["cdf", 
                                                        "pdf", 
                                                        {"ratio_above_threshold":{"elevation_threshold": [10,20,30]}}])

You can see that these new vector metrics are now available.

In [ ]:
stats_metrics = stats_dataset.get_classification_layer_metrics(classification_layer="global")
print(stats_metrics)

Get cdf value

In [ ]:
cdf = stats_dataset.get_classification_layer_metric(classification_layer = 'global', metric="cdf")

Plot cdf

In [ ]:
plot_cdf_pdf(cdf, "cdf")

Get pdf

In [ ]:
pdf = stats_dataset.get_classification_layer_metric(classification_layer = 'global', metric="pdf")

Plot pdf

In [ ]:
plot_cdf_pdf(pdf, "pdf")

Get the pourcentage of pixel above altitude threshold (ratio_above_threshold)

In [ ]:
rat = stats_dataset.get_classification_layer_metric(classification_layer="global", metric="ratio_above_threshold")

Plot ratio_above_treshold

In [ ]:
plot_ratio_above_threshold(rat, "Ratio above threshold")

##### 2.3.2. Computing the slope on computed altitudes differences

It is to be noticed that if two DEMs are defined as inputs, then the slope will be computed on both input DEMs separately, and not in the difference between both.

In [ ]:
plot_layers(stats_processing_, 'ref', altitude_diff, 'Slope layers on altitude diff', 'Slope0')

#### 2.3.3. Computing the segmentation on computed altitudes differences

In [ ]:
plot_layers(stats_processing_, 'ref', altitude_diff, 'Segmentation layers on altitude diff', 'Status')

#### 2.3.4. Fusion

In [ ]:
plot_layers(stats_processing_, 'ref', altitude_diff, 'Fusion from segmentation and slope on altitude diff', 'Fusion0')

#### 3. The cross classification and the modes

As shown in previous section, demcompare will classify stats according to classification layers and classification layer masks. Whenever a classification layer is given for both DEMs, then it can be possible to observe the metrics for pixels whose classification (segmentation for example) is the same between both DEM or not. These observations are available through what we call mode

Three mode are available : 
* standard : Within this mode all valid pixels are considered. 
* intersection and exclusion : These modes are only available if both DEMs (ref and sec) where classified by the same classification layer.
    * intersection :  is the standard mode where only the pixels sharing the same label for both DEMs classification layers are kept.
    * exclusion : is the intersection one complementary

For instance, the user can get mean metric results for slope layer and mode intersection

In [ ]:
mean = stats_dataset.get_classification_layer_metric(
            classification_layer="Slope0", metric="mean", mode="intersection"
        )
for cpt, i in enumerate(ranges_slope):
    print(f"In intersection mode mean for classe {ranges_slope[cpt]} is equal to {mean[cpt]}")